In [6]:
import os
import keras
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.applications import InceptionResNetV2

from skimage import transform
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
"""
    Preprocess image, normalizing and resizing it

    :param frame: RGBA frame
"""    
def preprocess_image(frame):
    
    # Normalize Pixel Values
    normalized_frame = frame/255.0 - 0.5
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, IMAGE_PP_SIZE)
    
    # Create a 3-Channel image
    final_image = np.dstack((preprocessed_frame, preprocessed_frame, preprocessed_frame))
    
    return final_image

"""
    Create 2D label list from 1D list
    
    :param labels: 1D label list
"""

def make_labels(labels):
    np_labels = np.zeros((len(labels), 16))
    
    for i in range(len(labels)):
        np_labels[i, labels[i]] = 1
    
    return np_labels

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def choices(l, k=1):
    new_list = []
    for i in range(k):
        new_list.append(random.choice(l))
    return new_list

In [12]:
IMAGE_PP_SIZE = [150, 150]
EPOCHS = 1
BATCH_SIZE = 100
TRAIN_STEP = 500
VAL_SIZE = 100

RELOAD_MODEL = False

In [16]:
if RELOAD_MODEL:
    try:
        json_file = open('model_imgnet.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights("model_imgnet.h5")
    except FileNotFoundError:
        conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))
        conv_base.traiable = False
        model = Sequential()
        model.add(conv_base)
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(Dense(1024, activation='relu'))
        model.add(Dense(512, activation='relu'))
        model.add(Dense(16, activation='softmax'))
else:
    conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))
    conv_base.traiable = False
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [17]:

#
# Define train, validation and test lists
#

train_labels = []
val_labels = []
test_labels = []

train_input = []
val_input = []
test_input = []

#
# Load validation dataset
#

folder = "/media/matheusmortatti/External/rvl-cdip"
labels_file = open(folder + "/labels/val.txt")

val_file = []
for line in labels_file:
    sp = line.split()
    val_file.append(sp)
    
#
# Open training list file and start training
#

labels_file = open(folder + "/labels/train.txt")
trained = 0
train_size = file_len(folder + "/labels/train.txt")

for i in range(EPOCHS):
    for line in labels_file:
        sp = line.split()
        pp_img = preprocess_image(imread(folder + "/images/" + sp[0]))

        train_input.append(pp_img)
        train_labels.append(int(sp[1]))

        if len(train_input) >= TRAIN_STEP:
            
            trained += len(train_input)

            #
            # Choose a subset of the validation data
            #

            ss_val = choices(val_file, k=VAL_SIZE)

            for v in ss_val:
                val_input.append(preprocess_image(imread(folder + "/images/" + v[0])))
                val_labels.append(int(v[1]))

            x_val = np.asarray(val_input)
            y_val = make_labels(val_labels)

            #
            # Create training data
            #

            x_train = np.asarray(train_input)
            y_train = make_labels(train_labels)

            model.fit(x_train, y_train,
                      batch_size=BATCH_SIZE,
                      epochs=1,
                      verbose=1,
                      validation_data=(x_val, y_val))

            #
            # Reset lists for next iteration
            #

            train_input = []
            train_labels = []
            val_input = []
            val_labels = []
            
            print("trained: " + str(trained) + " / " + str(train_size))


/home/matheusmortatti/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/matheusmortatti/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Train on 500 samples, validate on 100 samples
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[100,192,7,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node inception_resnet_v2_2/conv2d_502/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@train...kpropInput"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](inception_resnet_v2_2/activation_501/Relu, conv2d_502/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics_2/acc/Mean/_30293}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_47957_metrics_2/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"

In [ ]:
model_json = model.to_json()
with open("model_imgnet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_imgnet.h5")